In [229]:
import pandas as pd
import numpy as np
from scipy import stats

In [230]:
#Viewing raw CSV file initially
players = pd.read_csv("players.csv")
players.head()

,index,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,1,abdulza01,"April 7, 1946","Brooklyn, New York",1.2,42.8,NaN,72.8,505,15.1,...,5th overall,1st round,Cincinnati Royals,1968,6-9,"John Jay in Brooklyn, New York",Zaid Abdul-Aziz,Power Forward and Center,Right,235lb
2,2,abdulka01,"April 16, 1947","New York, New York",3.6,55.9,5.6,72.1,1560,24.6,...,1st overall,1st round,Milwaukee Bucks,1969,7-2,"Power Memorial in New York, New York",Kareem Abdul-Jabbar,Center,Right,225lb
3,3,abdulma02,"March 9, 1969","Gulfport, Mississippi",3.5,44.2,35.4,90.5,586,15.4,...,3rd overall,1st round,Denver Nuggets,1990,6-1,"Gulfport in Gulfport, Mississippi",Mahmoud Abdul-Rauf,Point Guard,Right,162lb
4,4,abdulta01,"November 3, 1974","Maisons Alfort, France",1.1,41.7,23.7,70.3,236,11.4,...,11th overall,1st round,Sacramento Kings,1997,6-6,"Lycee Aristide Briand in Evreux, France",Tariq Abdul-Wahad,Shooting Guard,Right,223lb


In [231]:
# dropping cols not important to analysis we are conducting
players = players.drop(['birthDate', 'birthPlace', 'college', 'draft_team', 'draft_year', 'height', 'draft_pick', 'draft_round',
                                    'highSchool', 'weight', 'shoots'], axis=1)
players.rename(columns={'_id':'player_id'}, inplace=True)
players.head()

,index,player_id,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,name,position
0,0,abdelal01,0.3,50.2,0.0,70.1,256,13.0,5.7,3.3,4.8,50.2,Alaa Abdelnaby,Power Forward
1,1,abdulza01,1.2,42.8,NaN,72.8,505,15.1,9.0,8.0,17.5,NaN,Zaid Abdul-Aziz,Power Forward and Center
2,2,abdulka01,3.6,55.9,5.6,72.1,1560,24.6,24.6,11.2,273.4,55.9,Kareem Abdul-Jabbar,Center
3,3,abdulma02,3.5,44.2,35.4,90.5,586,15.4,14.6,1.9,25.2,47.2,Mahmoud Abdul-Rauf,Point Guard
4,4,abdulta01,1.1,41.7,23.7,70.3,236,11.4,7.8,3.3,3.5,42.2,Tariq Abdul-Wahad,Shooting Guard


In [232]:
#Cleaning the position columns and dropping null values
players['position'] = players['position'].str.split(' and' )
player_positions = players.explode('position')
player_positions.dropna(inplace=True)
player_positions

,index,player_id,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,name,position
0,0,abdelal01,0.3,50.2,0.0,70.1,256,13.0,5.7,3.3,4.8,50.2,Alaa Abdelnaby,Power Forward
2,2,abdulka01,3.6,55.9,5.6,72.1,1560,24.6,24.6,11.2,273.4,55.9,Kareem Abdul-Jabbar,Center
3,3,abdulma02,3.5,44.2,35.4,90.5,586,15.4,14.6,1.9,25.2,47.2,Mahmoud Abdul-Rauf,Point Guard
4,4,abdulta01,1.1,41.7,23.7,70.3,236,11.4,7.8,3.3,3.5,42.2,Tariq Abdul-Wahad,Shooting Guard
5,5,abdursh01,2.5,47.2,29.7,81.0,830,19.0,18.1,7.5,71.2,47.9,Shareef Abdur-Rahim,Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4678,4678,zimmest01,0.2,32.3,-,60.0,19,7.3,1.2,1.8,0.0,32.3,Stephen Zimmerman,Center
4679,4679,zipsepa01,0.8,37.1,33.5,76.9,98,6.1,4.7,2.6,0.0,44.8,Paul Zipser,Small Forward
4680,4680,zizican01,0.6,58.3,-,70.8,91,17.5,6.4,4.2,3.0,58.3,Ante Zizic,Center
4681,4681,zoetji01,0.1,20.0,-,-,7,-0.8,0.3,1.1,-0.1,20.0,Jim Zoet,Center


In [233]:
#Viewing the raw salaries CSV file
salaries = pd.read_csv("salaries_1985to2018.csv")
salaries.head()

,index,league,player_id,salary,season,season_end,season_start,team
0,0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [234]:
# season category redundant, all leagues are also NBA (checked by value_counts()), so we can drop it too
salaries = salaries.drop(['league', 'season', 'season_end','index'], axis=1)
salaries['salary'] = salaries['salary'].astype(int)
salaries['season_start'] = pd.to_datetime(salaries['season_start'], format='%Y')
def normalize_by_year(x):
    # x is a DataFrame of group values
    x['salary'] /= x['salary'].max()
    return x
salaries = salaries.groupby('season_start', group_keys=False).apply(normalize_by_year)
salaries.rename(columns={'salary':'Normalized Salary'}, inplace=True)
salaries = salaries.groupby('player_id', group_keys=False)['Normalized Salary'].max()
salaries

/var/folders/z1/542vmcmd77n93176sv6nxkrw0000gn/T/ipykernel_56809/2606149918.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  salaries = salaries.groupby('season_start', group_keys=False).apply(normalize_by_year)


player_id
abdelal01    0.140244
abdulka01    0.923077
abdulma02    0.412238
abdulta01    0.243714
abdursh01    0.528046
               ...   
zidekge01    0.032205
zimmest01    0.030681
zipsepa01    0.037846
zizican01    0.047436
zubaciv01    0.037846
Name: Normalized Salary, Length: 2408, dtype: float64

In [235]:
historical = pd.merge(player_positions, salaries, how='right', on='player_id')
historical = historical.drop('index', axis=1)
historical

,player_id,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,name,position,Normalized Salary
0,abdelal01,0.3,50.2,0.0,70.1,256,13.0,5.7,3.3,4.8,50.2,Alaa Abdelnaby,Power Forward,0.140244
1,abdulka01,3.6,55.9,5.6,72.1,1560,24.6,24.6,11.2,273.4,55.9,Kareem Abdul-Jabbar,Center,0.923077
2,abdulma02,3.5,44.2,35.4,90.5,586,15.4,14.6,1.9,25.2,47.2,Mahmoud Abdul-Rauf,Point Guard,0.412238
3,abdulta01,1.1,41.7,23.7,70.3,236,11.4,7.8,3.3,3.5,42.2,Tariq Abdul-Wahad,Shooting Guard,0.243714
4,abdursh01,2.5,47.2,29.7,81.0,830,19.0,18.1,7.5,71.2,47.9,Shareef Abdur-Rahim,Center,0.528046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,zidekge01,0.2,40.8,25.0,78.3,135,8.7,3.4,2.1,0.8,40.9,George Zidek,Center,0.032205
3337,zimmest01,0.2,32.3,-,60.0,19,7.3,1.2,1.8,0.0,32.3,Stephen Zimmerman,Center,0.030681
3338,zipsepa01,0.8,37.1,33.5,76.9,98,6.1,4.7,2.6,0.0,44.8,Paul Zipser,Small Forward,0.037846
3339,zizican01,0.6,58.3,-,70.8,91,17.5,6.4,4.2,3.0,58.3,Ante Zizic,Center,0.047436


In [236]:
historical.head()
historical['position'] = historical['position'].astype('string')
historical['position'] = historical['position'].str.replace(r'\[|\]', '', regex=True)
historical['position'] = historical['position'].str.strip("'")
historical.head()

,player_id,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,name,position,Normalized Salary
0,abdelal01,0.3,50.2,0.0,70.1,256,13.0,5.7,3.3,4.8,50.2,Alaa Abdelnaby,Power Forward,0.140244
1,abdulka01,3.6,55.9,5.6,72.1,1560,24.6,24.6,11.2,273.4,55.9,Kareem Abdul-Jabbar,Center,0.923077
2,abdulma02,3.5,44.2,35.4,90.5,586,15.4,14.6,1.9,25.2,47.2,Mahmoud Abdul-Rauf,Point Guard,0.412238
3,abdulta01,1.1,41.7,23.7,70.3,236,11.4,7.8,3.3,3.5,42.2,Tariq Abdul-Wahad,Shooting Guard,0.243714
4,abdursh01,2.5,47.2,29.7,81.0,830,19.0,18.1,7.5,71.2,47.9,Shareef Abdur-Rahim,Center,0.528046


In [237]:
historical.dtypes

player_id                    object
career_AST                  float64
career_FG%                   object
career_FG3%                  object
career_FT%                   object
career_G                      int64
career_PER                   object
career_PTS                  float64
career_TRB                   object
career_WS                    object
career_eFG%                  object
name                         object
position             string[python]
Normalized Salary           float64
dtype: object

In [239]:
#Cleaning the data- adjusting dtypes to be workable

historical = historical[historical['career_FG%'] != '-']
historical['career_FG%'] = historical['career_FG%'].astype(float)

historical = historical[historical['career_FG3%'] != '-']
historical['career_FG3%'] = historical['career_FG3%'].astype(float)

historical = historical[historical['career_FT%'] != '-']
historical['career_FT%'] = historical['career_FT%'].astype(float)

historical = historical[historical['career_PER'] != '-']
historical['career_PER'] = historical['career_PER'].astype(float)

historical = historical[historical['career_PTS'] != '-']
historical['career_PTS'] = historical['career_PTS'].astype(float)
 
historical = historical[historical['career_WS'] != '-']
historical['career_WS'] = historical['career_WS'].astype(float)

historical = historical[historical['career_eFG%'] != '-']
historical['career_eFG%'] = historical['career_eFG%'].astype(float)

historical = historical[historical['career_TRB'] != '-']
historical['career_TRB'] = historical['career_TRB'].astype(float)

historical['player_id'] = historical['player_id'].astype('string')

historical['name'] = historical['name'].astype('string')
historical

,player_id,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,career_TRB,career_WS,career_eFG%,name,position,Normalized Salary
0,abdelal01,0.3,50.2,0.0,70.1,256,13.0,5.7,3.3,4.8,50.2,Alaa Abdelnaby,Power Forward,0.140244
1,abdulka01,3.6,55.9,5.6,72.1,1560,24.6,24.6,11.2,273.4,55.9,Kareem Abdul-Jabbar,Center,0.923077
2,abdulma02,3.5,44.2,35.4,90.5,586,15.4,14.6,1.9,25.2,47.2,Mahmoud Abdul-Rauf,Point Guard,0.412238
3,abdulta01,1.1,41.7,23.7,70.3,236,11.4,7.8,3.3,3.5,42.2,Tariq Abdul-Wahad,Shooting Guard,0.243714
4,abdursh01,2.5,47.2,29.7,81.0,830,19.0,18.1,7.5,71.2,47.9,Shareef Abdur-Rahim,Center,0.528046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3334,zellety01,0.9,50.9,28.6,76.4,412,14.9,7.0,4.4,17.1,51.1,Tyler Zeller,Center,0.258369
3335,zhizhwa01,0.3,41.7,38.5,73.5,137,14.2,4.4,1.7,2.1,48.5,Wang Zhizhi,Center,0.075397
3336,zidekge01,0.2,40.8,25.0,78.3,135,8.7,3.4,2.1,0.8,40.9,George Zidek,Center,0.032205
3338,zipsepa01,0.8,37.1,33.5,76.9,98,6.1,4.7,2.6,0.0,44.8,Paul Zipser,Small Forward,0.037846
